In [7]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

In [8]:
IMG_SIZE = 224
DATA_DIR = 'dataset'
CATEGORIES = ['NORMAL', 'PNEUMONIA']

In [10]:
def load_data():
    data = []
    for category in CATEGORIES:
        folder = os.path.join(DATA_DIR, category)
        label = CATEGORIES.index(category)
        for img in tqdm(os.listdir(folder)):
            path = os.path.join(folder, img)
            try:
                image = cv2.imread(path)
                image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                data.append([image, label])
            except:
                pass
    np.random.shuffle(data)
    X, y = zip(*data)
    return np.array(X)/255.0, np.array(y)

X, y = load_data()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

base_model = MobileNetV2(weights=None, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)


100%|██████████| 100/100 [00:00<00:00, 942.34it/s]


In [11]:
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

model.save("model/model.h5")

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 411ms/step - accuracy: 0.4313 - loss: 0.6932 - val_accuracy: 0.5250 - val_loss: 0.6931
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 266ms/step - accuracy: 0.4812 - loss: 0.6932 - val_accuracy: 0.4750 - val_loss: 0.6932
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step - accuracy: 0.5063 - loss: 0.6931 - val_accuracy: 0.4750 - val_loss: 0.6932
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step - accuracy: 0.5063 - loss: 0.6931 - val_accuracy: 0.4750 - val_loss: 0.6932
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 260ms/step - accuracy: 0.5063 - loss: 0.6931 - val_accuracy: 0.4750 - val_loss: 0.6932
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 261ms/step - accuracy: 0.5063 - loss: 0.6931 - val_accuracy: 0.4750 - val_loss: 0.6932
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 261ms/step - accuracy: 0.5063 - loss: 0.6931 - val_accuracy: 0.4750 - val_loss: 0.6932
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step - accuracy: 0.5063 - loss: 0.6931 - val_accuracy: 0.4750 - val_loss: